<a href="https://colab.research.google.com/github/smtolo/EDSA/blob/master/Classification_Hackathon_1074.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classification Hackathon

For the specifications for today's Hackathon use the slides linked below. Just note a couple things before you start:
* Use your full name and ```_EDSA``` as your Zindi username.
* The dataset for this challenge is very large and will take a long time to process. In order to use your time wisely, only use a small subset of the data to figure out how to solve this challenge, and once you're happy with that, train your model with the entire dataset.
* This Zindi challenge is tough. This will be taken into account when the supervisors mark your work. Do not worry too much about your placement on the leaderboard. In the Regression Hackathon ```laura_the_explorer``` was in first place but is outside the top 100 in this challenge
* To submit your Hackathon to Athena, zip your notebook and your submission csv file, and upload that here. Note that your report card will say you have 100% once you submit your file.
* Please attach the *Honour code* (below) cell to your notebook. 

Further instructions found on these slides: https://docs.google.com/presentation/d/1AbVndI5aOd27Jm0E1qNoYzRtWiZ6-DE3BDE0djGxzIk/edit?usp=sharing

** Good luck! **

## Honour Code
I Sihle, YOUR Mtolo, confirm - by submitting my - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

In [0]:
import numpy as np
import pandas as pd
import io

In [514]:
df_train = pd.read_csv('training.csv')
df_train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


In [515]:
df_test = pd.read_csv('test (1).csv')
df_test.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [0]:
df_train['is_train'] = True
df_test['is_train'] = False

In [517]:
df = df_train.append(df_test)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [518]:
len(df)

140681

In [0]:
test_id = df_test['TransactionId']

In [520]:
df['Transaction_year'] = df['TransactionStartTime'].str[:4].astype(int)

# df['Transaction_month'] = df['TransactionStartTime'].str[5:7].astype(int)
# df['Transaction_day'] = df['TransactionStartTime'].str[8:10].astype(int)
# df['Transaction_hour'] = df['TransactionStartTime'].str[11:13].astype(int)
df['AccountId'] = df['AccountId'].str[10:].astype(int)
df['BatchId'] = df['BatchId'].str[8:].astype(int)
df['ChannelId'] = df['ChannelId'].str[10:].astype(int)
df['CustomerId'] = df['CustomerId'].str[11:].astype(int)
df['ProductId'] = df['ProductId'].str[10:].astype(int)
df['ProviderId'] = df['ProviderId'].str[11:].astype(int)
df['SubscriptionId'] = df['SubscriptionId'].str[15:].astype(int)

df

,AccountId,Amount,BatchId,ChannelId,CountryCode,CurrencyCode,CustomerId,FraudResult,PricingStrategy,ProductCategory,ProductId,ProviderId,SubscriptionId,TransactionId,TransactionStartTime,Value,is_train,Transaction_year
0,3957,1000.0,36123,3,256,UGX,4406,0.0,2,airtime,10,6,887,TransactionId_76871,2018-11-15T02:18:49Z,1000,True,2018
1,4841,-20.0,15642,2,256,UGX,4406,0.0,2,financial_services,6,4,3829,TransactionId_73770,2018-11-15T02:19:08Z,20,True,2018
2,4229,500.0,53941,3,256,UGX,4683,0.0,2,airtime,1,6,222,TransactionId_26203,2018-11-15T02:44:21Z,500,True,2018
3,648,20000.0,102363,3,256,UGX,988,0.0,2,utility_bill,21,1,2185,TransactionId_380,2018-11-15T03:32:55Z,21800,True,2018
4,4841,-644.0,38780,2,256,UGX,988,0.0,2,financial_services,6,4,3829,TransactionId_28195,2018-11-15T03:34:21Z,644,True,2018
5,1078,2000.0,25954,3,256,UGX,1432,0.0,2,airtime,3,6,4238,TransactionId_23223,2018-11-15T03:35:10Z,2000,True,2018
6,2442,10000.0,118460,3,256,UGX,2858,0.0,4,airtime,3,5,1980,TransactionId_118063,2018-11-15T03:44:31Z,10000,True,2018
7,4841,-500.0,38561,2,256,UGX,2858,0.0,2,financial_services,6,4,3829,TransactionId_100640,2018-11-15T03:45:13Z,500,True,2018
8,272,500.0,93774,3,256,UGX,598,0.0,2,airtime,10,6,4731,TransactionId_51905,2018-11-15T04:14:59Z,500,True,2018
9,710,600.0,82409,3,256,UGX,1053,0.0,2,financial_services,15,1,920,TransactionId_130161,2018-11-15T04:31:48Z,600,True,2018


In [0]:
df.drop(['TransactionStartTime','CurrencyCode','TransactionId','CustomerId', 'AccountId', 'CountryCode','SubscriptionId', 'ProviderId'], inplace=True, axis=1)

In [522]:
df.head()

,Amount,BatchId,ChannelId,FraudResult,PricingStrategy,ProductCategory,ProductId,Value,is_train,Transaction_year
0,1000.0,36123,3,0.0,2,airtime,10,1000,True,2018
1,-20.0,15642,2,0.0,2,financial_services,6,20,True,2018
2,500.0,53941,3,0.0,2,airtime,1,500,True,2018
3,20000.0,102363,3,0.0,2,utility_bill,21,21800,True,2018
4,-644.0,38780,2,0.0,2,financial_services,6,644,True,2018


In [0]:
df = pd.get_dummies(df)

In [0]:
df_train = df[df['is_train'] == True]
df_test = df[df['is_train'] == False]

In [525]:
df_train.drop('is_train', axis=1, inplace =True)
df_test.drop('is_train', axis=1, inplace =True)
df_train['FraudResult'] = df_train['FraudResult'].astype(int)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
X = df_train.drop('FraudResult', axis=1)
y = df_train['FraudResult']

testing_X = df_test.drop('FraudResult', axis=1)

In [0]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [0]:
sm = SMOTE()

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# grid = GridSearchCV(scoring=f1_score, estimator=RandomForestClassifier, param_grid={})

In [0]:
rnd = RandomForestClassifier()

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)

In [532]:
rnd.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
y_pred = rnd.predict(X_test)

In [0]:
from sklearn.metrics import accuracy_score, f1_score, precision_score

In [535]:
accuracy_score(y_test, y_pred)


0.9998432028432551

In [536]:
f1_score(y_test, y_pred)

0.9577464788732395

In [537]:
precision_score(y_test, y_pred)

0.9714285714285714

In [0]:
final_model = rnd.predict(testing_X)

In [0]:
df_sub = pd.DataFrame({'TransactionId':test_id, 'FraudResult':final_model})

df_sub.set_index('TransactionId', inplace=True)

In [540]:
df_sub.to_csv('Submissions12.csv')

pd.read_csv('Submissions12.csv')

,TransactionId,FraudResult
0,TransactionId_50600,0
1,TransactionId_95109,0
2,TransactionId_47357,0
3,TransactionId_28185,0
4,TransactionId_22140,0
5,TransactionId_134338,0
6,TransactionId_109096,0
7,TransactionId_14249,0
8,TransactionId_69896,0
9,TransactionId_91468,0
